做一个动态链接库

```sh
cd ../skynet-master 

# 编译
cc -g -O2 -Wall -fPIC -dynamiclib -Wl,-undefined,dynamic_lookup -I3rd/lua  -o libskynet.so skynet-src/skynet_main.c skynet-src/skynet_handle.c skynet-src/skynet_module.c skynet-src/skynet_mq.c skynet-src/skynet_server.c skynet-src/skynet_start.c skynet-src/skynet_timer.c skynet-src/skynet_error.c skynet-src/skynet_harbor.c skynet-src/skynet_env.c skynet-src/skynet_monitor.c skynet-src/skynet_socket.c skynet-src/socket_server.c skynet-src/malloc_hook.c skynet-src/skynet_daemon.c skynet-src/skynet_log.c 3rd/lua/liblua.a -Iskynet-src -I3rd/jemalloc/include/jemalloc   -lpthread -lm -ldl -DNOUSE_JEMALLOC

```

In [2]:
from cffi import FFI
ffi = FFI()

In [3]:
ffi.cdef("""

const char * skynet_getenv(const char *key);
void skynet_setenv(const char *key, const char *value);

void skynet_env_init();
""", override=True)



In [4]:
lib = ffi.dlopen('../skynet-master/libskynet.so')


In [5]:
lib.skynet_env_init()

In [14]:
# 定义变量
k = ffi.new("char[]", b"v1")
v = ffi.new("char[]", b"100")


In [17]:
k = b"host"
v = b"localhost"
lib.skynet_setenv(k, v)

In [18]:
ffi.string(lib.skynet_getenv(k))

b'localhost'

In [15]:
# 赋值
for i in range(10):
    k = b"V%d"%i
    v = b"%d"%i
    lib.skynet_setenv(k, v)
    


In [16]:
# 获得值
for i in range(10):
    k = b"V%d"%i
    print(ffi.string(lib.skynet_getenv(k)))

b'0'
b'1'
b'2'
b'3'
b'4'
b'5'
b'6'
b'7'
b'8'
b'9'


## 测试 setenv 时,  key已经存在
```C
/** 向lua虚拟机中 */
void 
skynet_setenv(const char *key, const char *value) {
	SPIN_LOCK(E)
	
	lua_State *L = E->L;
	lua_getglobal(L, key); // 将 key对应的值压入栈顶
	assert(lua_isnil(L, -1)); // key 对应的值应该为 nil. 如果不为 nil?
	lua_pop(L,1);
	lua_pushstring(L,value);
	lua_setglobal(L,key);

	SPIN_UNLOCK(E)
}

```

In [19]:
k = b"key_t2"
v = b"value of t2"
lib.skynet_setenv(k, v)

使用原有 k 继续

In [ ]:
v = b"value 2 of t2"
lib.skynet_setenv(k, v)


执行后, jupyter 提示 "服务似乎挂掉了,但是会立刻重启的."